# Imports and packages

In [1]:
## helpful packages

import warnings
warnings.filterwarnings("ignore")

import pandas as pd
import numpy as np
import random
import re
import string

## nltk imports
import nltk
### uncomment and run these lines if you haven't downloaded relevant nltk add-ons yet
#nltk.download('averaged_perceptron_tagger')
#nltk.download('stopwords')
from nltk import pos_tag
from nltk.tokenize import word_tokenize, wordpunct_tokenize
from nltk.stem.snowball import SnowballStemmer
from nltk.stem.porter import PorterStemmer
from nltk.corpus import stopwords

## spacy imports
import spacy
### uncomment and run the below line if you haven't loaded the en_core_web_sm library yet
#! python3 -m spacy download en_core_web_sm
import en_core_web_sm
nlp = en_core_web_sm.load()

## vectorizer
from sklearn.feature_extraction.text import CountVectorizer

## sentiment
from vaderSentiment.vaderSentiment import SentimentIntensityAnalyzer

## lda
from gensim import corpora
import gensim

## repeated printouts and wide-format text
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"
pd.set_option('display.max_colwidth', None)

# Text analysis of Department of Justice (DOJ) press releases (50 points total)

- For background:

    - DOJ is the federal law enforcement agency responsible for federal prosecutions; this contrasts with the local prosecutions in the Cook County dataset we analyzed earlier. Here's a short explainer on which crimes get prosecuted federally versus locally: https://www.criminaldefenselawyer.com/resources/criminal-defense/federal-crime/state-vs-federal-crimes.htm#:~:text=Federal%20criminal%20prosecutions%20are%20handled,of%20state%20and%20local%20law. 
    - Here's the Kaggle that contains the data: https://www.kaggle.com/jbencina/department-of-justice-20092018-press-releases 
    - Here's the code the dataset creator used to scrape those press releases here if you're interested: https://github.com/jbencina/dojreleases
    
- See here for a codebook: https://docs.google.com/spreadsheets/d/1UopmSvFGrwJvz_c3Plh32Yxkqwff64oS_CcpfATOV8k/edit?usp=sharing

In [2]:
## first, unzip the combined.json.zip file
## then, run this code to load the unzipped json file and convert to a dataframe
## and convert some of the attributes from lists to values
## make sure to change the pathname if you need to
doj = pd.read_json("combined.json", lines = True)

## due to json, topics are in a list so remove them and concatenate with ;
doj['topics_clean'] = ["; ".join(topic) 
                      if len(topic) > 0 else "No topic" 
                      for topic in doj.topics]

## similarly with components
doj['components_clean'] = ["; ".join(comp) 
                           if len(comp) > 0 else "No component" 
                           for comp in doj.components]

## drop older columns from data
doj = doj[['id', 'title', 'contents', 'date', 'topics_clean', 
           'components_clean']].copy()

doj.head()


id  \
0     None   
1  12-919    
2  11-1002   
3   10-015   
4   18-898   

                                                                                                          title  \
0                                                                  Convicted Bomb Plotter Sentenced to 30 Years   
1                              $1 Million in Restitution Payments Announced to Preserve North Carolina Wetlands   
2                 $1 Million Settlement Reached for Natural Resource Damages at Superfund Site in Massachusetts   
3                                          10 Las Vegas Men Indicted \r\nfor Falsifying Vehicle Emissions Tests   
4  $100 Million Settlement Will Speed Cleanup Work at Centredale Manor Superfund Site in North Providence, R.I.   

                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                

## 1. Tagging and sentiment scoring (16 points)

Focus on the following press release: `id` == "17-1204" about this pharmaceutical kickback prosecution: https://www.forbes.com/sites/michelatindera/2017/11/16/fentanyl-billionaire-john-kapoor-to-plead-not-guilty-in-opioid-kickback-case/?sh=21b8574d6c6c 

The `contents` column is the one we're treating as a document. You may need to to convert it from a pandas series to a single string.

We'll call the raw string of this press release `pharma`

In [3]:
contents = doj[doj['id'] == '17-1204']['contents']
pharma = contents.to_string(index=False)
type(pharma)
len(pharma)

str

9252

### 1.1 part of speech tagging (3 points)

A. Preprocess the `pharma` press release to remove all punctuation / digits (so can use `.isalpha()` to subset)

B. With the preprocessed press release from part A, use the part of speech tagger within nltk to tag all the words in that one press release with their part of speech. 

C. Using the output from B, extract the adjectives and sort those adjectives from most occurrences to fewest occurrences. Print a dataframe with the 5 most frequent adjectives and their counts in the `pharma` release. See here for a list of the names of adjectives within nltk: https://pythonprogramming.net/natural-language-toolkit-nltk-part-speech-tagging/

**Resources**:

- Documentation for .isalpha(): https://www.w3schools.com/python/ref_string_isalpha.asp
- `process_step1` function here has an example of tokenizing and filtering to words where .isalpha() is true: 
https://github.com/rebeccajohnson88/PPOL564_slides_activities/blob/main/activities/fall_22/solutions/09_textasdata_partII_topicmodeling_solution.ipynb 
- Part of speech tagging section of this code: 
https://github.com/rebeccajohnson88/PPOL564_slides_activities/blob/main/activities/fall_22/solutions/08_textasdata_partI_textmining_solutions.ipynb



In [4]:
tokens = [word for word in word_tokenize(pharma) if word.isalpha()]
# tokens

In [5]:
tokens_pos = pos_tag(tokens) # assign parts of speech for each token
# tokens_pos

In [6]:
from collections import Counter

# [element for element in list_of_elements]

# output = []
# for element in list_of_elements:
#   if element > 5:
#     if element < 99:
#        output.append(element)

tokens_adj = [pos for pos in tokens_pos if pos[1] in set(["JJ", "JJR", "JJS"]) and len(pos[0]) > 1]
adj_count = Counter(tokens_adj)
adj_count.most_common(5)

[(('former', 'JJ'), 8),
 (('opioid', 'JJ'), 5),
 (('nationwide', 'JJ'), 4),
 (('other', 'JJ'), 3),
 (('addictive', 'JJ'), 3)]

## 1.2 named entity recognition (3 points)



A. Using the original `pharma` press release (so the one before stripping punctuation/digits), use spaCy to extract all named entities from the press release.

B. Print the unique named entities with the tag: `LAW`. Here's some background on what RICO means: https://www.justia.com/criminal/docs/rico/ 

**Resources**:
- For parts A and B: named entity recognition part of this code: 
https://github.com/rebeccajohnson88/PPOL564_slides_activities/blob/main/activities/fall_22/solutions/08_textasdata_partI_textmining_solutions.ipynb

In [7]:
spacy_doc = nlp(pharma)
spacy_ents = spacy_doc.ents
LAW = set([ent.text for ent in spacy_ents if ent.label_ == "LAW"])
LAW

{'RICO', 'the Controlled Substances Act'}

C. You want to extract the possible sentence lengths the CEO is facing; pull out the named entities with (1) the label `DATE` and (2) that contain the word "year" or "years". Print these named entities.

**Hint:**  
You may want to use the `re` module for the second part.

In [9]:
# %pip install session_info

In [10]:
# %pip install spacy==3.4.2

In [11]:
# import session_info
# session_info.show()

In [8]:
CEO = set([ent.text for ent in spacy_ents if ent.label_ == "DATE" and "year" in ent.text])
CEO

{'20 years', 'last year', 'three years'}

D. Parse the pharma string at the sentence level. Note that this involves more than just splitting on each `.`; for full credit, add at least one additional delimiter that marks the end of the sentence.

Then, using those sentences, pull and print the original sentences from the press releases where those year lengths are mentioned. Describe in your own words (1 sentence) what length of sentence (prison) and probation (supervised release) the CEO may be facing if convicted after this indictment (if there are multiple lengths mentioned describe the maximum). 

**Hint:**  
You may want to use re.search or re.findall 

**Resources**:

- re.search and re.findall examples here for filtering to ones containing year (multiple approaches; some need not involve `re`): 
https://github.com/rebeccajohnson88/PPOL564_slides_activities/blob/main/activities/fall_22/solutions/07_regex_solutions.ipynb


In [9]:
# This is how we do it in practice
LL = [[sent for sent in spacy_doc.sents if str(e) in str(sent)] for e in CEO]
{s for l in LL for s in l}

# To show we can do it the way we've been told
pharma_clean=pharma.replace(u'\xa0',u'')
re.findall(r'([^.]+?%s+[^.]*)'%'year',pharma_clean)

{ "More than 20,000 Americans died of synthetic opioid overdoses last year, and millions are addicted to opioids.,
 The charges of conspiracy to commit RICO and conspiracy to commit mail and wire fraud each provide for a sentence of no greater than 20 years in prison, three years of supervised release and a fine of $250,000, or twice the amount of pecuniary gain or loss.,
   The charges of conspiracy to violate the Anti-Kickback Law provide for a sentence of no greater than five years in prison, three years of supervised release and a $25,000 fine.}

['"More than 20,000 Americans died of synthetic opioid overdoses last year, and millions are addicted to opioids',
 'The charges of conspiracy to commit RICO and conspiracy to commit mail and wire fraud each provide for a sentence of no greater than 20 years in prison, three years of supervised release and a fine of $250,000, or twice the amount of pecuniary gain or loss',
 ' The charges of conspiracy to violate the Anti-Kickback Law provide for a sentence of no greater than five years in prison, three years of supervised release and a $25,000 fine']

A RICO conviction could bring 20 years prison time and three years probation; but an Anti-Kickback conviction could bring five years prison time and three years probation.

## 1.3 sentiment analysis  (10 points)

- Sentiment analysis section of this script: 
https://github.com/rebeccajohnson88/PPOL564_slides_activities/blob/main/activities/fall_22/solutions/08_textasdata_partI_textmining_solutions.ipynb


A. Subset the press releases to those labeled with one of three topics via `topics_clean`: Civil Rights, Hate Crimes, and Project Safe Childhood. We'll call this `doj_subset` going forward and it should have 717 rows.



In [10]:
doj_subset=doj.loc[(doj.topics_clean=='Civil Rights')|(doj.topics_clean=='Hate Crimes')|(doj.topics_clean=='Project Safe Childhood')].reset_index()
doj_subset.shape
doj_subset.loc[doj_subset.id=='14-248']
doj_subset.loc[doj_subset.id=='16-718']

(717, 7)

,index,id,title,contents,date,topics_clean,components_clean
13,329,14-248,Albuquerque Man Charged with Federal Hate Crime Related to Anti-Semitic Threats Against Businesswoman,"The Department of Justice announced that this morning John W. Ng, 58, of Albuquerque, N.M., made his initial appearance in federal court on a criminal complaint charging him with a hate crime offense. This charge is related to anti-Semitic threats Ng made against a Jewish woman who owns and operates the Nosh Jewish Delicatessen and Bakery in Albuquerque. Ng was arrested by the FBI on March 7, 2014, based on a criminal complaint alleging that he interfered with the victim’s federally protected rights by threatening her and interfering with her business because of her religion. According to the criminal complaint, between Jan. 22, 2014, and Feb. 8, 2014, Ng allegedly posted threatening anti-Semitic notes on and in the vicinity of the victim’s business. A criminal complaint merely establishes probable cause, and Ng is presumed innocent unless proven guilty. If convicted on the offense charged in the criminal complaint, Ng faces a maximum statutory penalty of one year in prison. This matter was investigated by the Albuquerque Division of the FBI and is being prosecuted by Assistant U.S. Attorney Mark T. Baker of the U.S. Attorney’s Office for the District of New Mexico and Trial Attorney AeJean Cha of the U.S. Department of Justice’s Civil Rights Division.",2014-03-10T00:00:00-04:00,Hate Crimes,Civil Rights Division; Civil Rights - Criminal Section


,index,id,title,contents,date,topics_clean,components_clean
632,11593,16-718,Three Mississippi Correctional Officers Indicted for Inmate Assault and Cover-Up,"In a nine-count indictment unsealed today, two Mississippi correctional officers were charged with beating an inmate and a third was charged with helping to cover it up. The indictment charged Lawardrick Marsher, 28, and Robert Sturdivant, 47, officers at Mississippi State Penitentiary, in Parchman, Mississippi, with a beating that included kicking, punching and throwing the victim to the ground. Marsher and Sturdivant were charged with violating the right of K.H., a convicted prisoner, to be free from cruel and unusual punishment. Sturdivant was also charged with failing to intervene while Marsher was punching and beating K.H. The indictment alleges that their actions involved the use of a dangerous weapon and resulted in bodily injury to the victim. A third officer, Deonte Pate, 23, was charged along with Marsher and Sturdivant for conspiring to cover up the beating. The indictment alleges that all three officers submitted false reports and that all three lied to the FBI. If convicted, Marsher and Sturdivant face a maximum sentence of 10 years in prison on the excessive force charges. Each of the three officers faces up to five years in prison on the conspiracy and false statement charges, and up to 20 years in prison on the false report charges. An indictment is merely an accusation, and the defendants are presumed innocent unless and until proven guilty. This case is being investigated by the FBI’s Jackson Division, with the cooperation of the Mississippi Department of Corrections. It is being prosecuted by Assistant U.S. Attorney Robert Coleman of the Northern District of Mississippi and Trial Attorney Dana Mulhauser of the Civil Rights Division’s Criminal Section. Marsher Indictment",2016-06-21T00:00:00-04:00,Civil Rights,"Civil Rights Division; Civil Rights - Criminal Section; USAO - Mississippi, Northern"


B. Write a function that takes one press release string as an input and:

- Removes named entities from each press release string (**Hint:** you may want to use `re.sub` with an or condition)
- Scores the sentiment of the entire press release using the `SentimentIntensityAnalyzer` and `polarity_scores`
- Returns the length-four (negative, positive, neutral, compound) sentiment dictionary (any order is fine)

Apply that function to each of the press releases in `doj_subset`. 

**Hints**: 

- I used a function + list comprehension to execute and it takes about 30 seconds on my local machine; if it's taking a very long time, you may want to check your code for inefficiencies. If you can't fix those, for partial credit on this part/full credit on remainder, you can take a small random sample of the 717


In [11]:
def sentiment_analysis(press_release_string):
    spacy_press_release_string = nlp(press_release_string)
    all_entities = [one_element.text for one_element in spacy_press_release_string.ents]
    long_statement = "|".join(all_entities).replace('(','').replace(')','')
    new_str = re.sub(long_statement, '', press_release_string)
    sent_obj = SentimentIntensityAnalyzer()
    sentiment = sent_obj.polarity_scores(new_str)
    return sentiment

sentiment_analysis(doj_subset.contents.iloc[11])

{'neg': 0.105, 'neu': 0.832, 'pos': 0.063, 'compound': -0.9153}

In [12]:
all_sentiment=[sentiment_analysis(press_release_string) for press_release_string in doj_subset.contents]
all_sentiment

[{'neg': 0.201, 'neu': 0.75, 'pos': 0.05, 'compound': -0.9931},
 {'neg': 0.136, 'neu': 0.795, 'pos': 0.07, 'compound': -0.9325},
 {'neg': 0.094, 'neu': 0.828, 'pos': 0.078, 'compound': -0.7579},
 {'neg': 0.126, 'neu': 0.79, 'pos': 0.084, 'compound': -0.9037},
 {'neg': 0.178, 'neu': 0.777, 'pos': 0.044, 'compound': -0.9864},
 {'neg': 0.147, 'neu': 0.8, 'pos': 0.052, 'compound': -0.987},
 {'neg': 0.161, 'neu': 0.76, 'pos': 0.078, 'compound': -0.9595},
 {'neg': 0.088, 'neu': 0.842, 'pos': 0.071, 'compound': -0.6597},
 {'neg': 0.105, 'neu': 0.834, 'pos': 0.061, 'compound': -0.9136},
 {'neg': 0.168, 'neu': 0.776, 'pos': 0.057, 'compound': -0.9801},
 {'neg': 0.216, 'neu': 0.741, 'pos': 0.043, 'compound': -0.9972},
 {'neg': 0.105, 'neu': 0.832, 'pos': 0.063, 'compound': -0.9153},
 {'neg': 0.085, 'neu': 0.848, 'pos': 0.067, 'compound': -0.6486},
 {'neg': 0.315, 'neu': 0.654, 'pos': 0.031, 'compound': -0.9951},
 {'neg': 0.175, 'neu': 0.758, 'pos': 0.067, 'compound': -0.9889},
 {'neg': 0.126, 'n

C. Add the four sentiment scores to the `doj_subset` dataframe to create a dataframe: `doj_subset_wscore`. Sort from highest to lowest neg score (so most negative to least negative) and print the `id`, `contents`, and `neg` columns of the two most negative press releases. 

Notes:

- Don't worry if your sentiment score differs slightly from our output on GitHub; differences in preprocessing can lead to diff scores

In [13]:
df_all_sentiment = pd.DataFrame.from_dict(all_sentiment)

doj_subset_wscore = pd.concat([doj_subset, df_all_sentiment], axis=1).sort_values(by='neg', ascending=False)
doj_subset_wscore.loc[:, ['id', 'contents', 'neg']].head(n=2)

,id,contents,neg
13,14-248,"The Department of Justice announced that this morning John W. Ng, 58, of Albuquerque, N.M., made his initial appearance in federal court on a criminal complaint charging him with a hate crime offense. This charge is related to anti-Semitic threats Ng made against a Jewish woman who owns and operates the Nosh Jewish Delicatessen and Bakery in Albuquerque. Ng was arrested by the FBI on March 7, 2014, based on a criminal complaint alleging that he interfered with the victim’s federally protected rights by threatening her and interfering with her business because of her religion. According to the criminal complaint, between Jan. 22, 2014, and Feb. 8, 2014, Ng allegedly posted threatening anti-Semitic notes on and in the vicinity of the victim’s business. A criminal complaint merely establishes probable cause, and Ng is presumed innocent unless proven guilty. If convicted on the offense charged in the criminal complaint, Ng faces a maximum statutory penalty of one year in prison. This matter was investigated by the Albuquerque Division of the FBI and is being prosecuted by Assistant U.S. Attorney Mark T. Baker of the U.S. Attorney’s Office for the District of New Mexico and Trial Attorney AeJean Cha of the U.S. Department of Justice’s Civil Rights Division.",0.315
632,16-718,"In a nine-count indictment unsealed today, two Mississippi correctional officers were charged with beating an inmate and a third was charged with helping to cover it up. The indictment charged Lawardrick Marsher, 28, and Robert Sturdivant, 47, officers at Mississippi State Penitentiary, in Parchman, Mississippi, with a beating that included kicking, punching and throwing the victim to the ground. Marsher and Sturdivant were charged with violating the right of K.H., a convicted prisoner, to be free from cruel and unusual punishment. Sturdivant was also charged with failing to intervene while Marsher was punching and beating K.H. The indictment alleges that their actions involved the use of a dangerous weapon and resulted in bodily injury to the victim. A third officer, Deonte Pate, 23, was charged along with Marsher and Sturdivant for conspiring to cover up the beating. The indictment alleges that all three officers submitted false reports and that all three lied to the FBI. If convicted, Marsher and Sturdivant face a maximum sentence of 10 years in prison on the excessive force charges. Each of the three officers faces up to five years in prison on the conspiracy and false statement charges, and up to 20 years in prison on the false report charges. An indictment is merely an accusation, and the defendants are presumed innocent unless and until proven guilty. This case is being investigated by the FBI’s Jackson Division, with the cooperation of the Mississippi Department of Corrections. It is being prosecuted by Assistant U.S. Attorney Robert Coleman of the Northern District of Mississippi and Trial Attorney Dana Mulhauser of the Civil Rights Division’s Criminal Section. Marsher Indictment",0.296


D. With the dataframe from part C, find the mean compound sentiment score for each of the three topics in `topics_clean` using group_by and agg.

E. Add a 1 sentence interpretation of why we might see the variation in scores (remember that compound is a standardized summary where -1 is most negative; +1 is most positive)


In [14]:
doj_subset_wscore.groupby('topics_clean').agg({'compound':np.mean})


,compound
topics_clean,
Civil Rights,-0.100690
Hate Crimes,-0.935284
Project Safe Childhood,-0.661069


This shows that the words describing Civil Rights abuses are only slightly less negative compared to those used with more severe crimes like child abuse or hate crimes.

# 2. Topic modeling (25 points)

For this question, use the `doj_subset_wscores` data that is restricted to civil rights, hate crimes, and project safe childhood and with the sentiment scores added


## 2.1 Preprocess the data by removing stopwords, punctuation, and non-alpha words (5 points)

A. Write a function that:

- Takes in a single raw string in the `contents` column from that dataframe
- Does the following preprocessing steps:

    - Converts the words to lowercase
    - Removes stopwords, adding the custom stopwords in the code cell below to the default stopwords list
    - Only retains alpha words (so removes digits and punctuation)
    - Only retains words 4 characters or longer
    - Uses the snowball stemmer from nltk to stem
    - Returns a joined preprocessed string (so if press release is something like "The CEO was indicted" it might return "ceo indict" 
    
B. Use `apply` or list comprehension to execute that function and create a new column in the data called `processed_text`. Note: there will be a deduction if your code uses a non-list comprehension for loop that uses append.
    
C. Print the `id`, `contents`, and `processed_text` columns for the following press releases:

id = 16-718 (this case: https://www.seattletimes.com/nation-world/doj-miami-police-reach-settlement-in-civil-rights-case/)

id = 16-217 (this case: https://www.wlbt.com/story/32275512/three-mississippi-correctional-officers-indicted-for-inmate-assault-and-cover-up/)
    
**Resources**:

- Here's code examples for the snowball stemmer: https://www.geeksforgeeks.org/snowball-stemmer-nlp/
- Here's code with topic modeling steps: 
https://github.com/rebeccajohnson88/PPOL564_slides_activities/blob/main/activities/fall_22/solutions/09_textasdata_partII_topicmodeling_solution.ipynb

In [15]:
custom_doj_stopwords = ["civil", "rights", "division", "department", "justice",
                        "office", "attorney", "district", "case", "investigation", "assistant",
                       "trial", "assistance", "assist"]

In [16]:
def text_process(raw_string):
    try:
        # Load stopwords list and augment with our own custom ones
        list_stopwords = stopwords.words("english")
        list_stopwords_new = list_stopwords + custom_doj_stopwords
        
        ## convert to lowercase
        lower = raw_string.lower()
        
        # initialize stemmer
        porter = PorterStemmer()

        nostop_listing = [word for word in wordpunct_tokenize(lower)
                          if word not in list_stopwords_new]
        clean_listing = [porter.stem(word) for word in nostop_listing
                        if word.isalpha() 
                        and len(word) > 3]
        clean_listing_str = " ".join(clean_listing)
        return(clean_listing_str)
    except:
        return("")

In [17]:
cleaned_listings = [text_process(raw_string) for raw_string in 
                   doj_subset_wscore.contents]
doj_subset_wscore['processed_text'] = cleaned_listings

In [18]:
doj_subset_wscore.loc[(doj_subset_wscore.id=='16-718')|(doj_subset_wscore.id=='16-217')][['id','contents','processed_text']]

,id,contents,processed_text
632,16-718,"In a nine-count indictment unsealed today, two Mississippi correctional officers were charged with beating an inmate and a third was charged with helping to cover it up. The indictment charged Lawardrick Marsher, 28, and Robert Sturdivant, 47, officers at Mississippi State Penitentiary, in Parchman, Mississippi, with a beating that included kicking, punching and throwing the victim to the ground. Marsher and Sturdivant were charged with violating the right of K.H., a convicted prisoner, to be free from cruel and unusual punishment. Sturdivant was also charged with failing to intervene while Marsher was punching and beating K.H. The indictment alleges that their actions involved the use of a dangerous weapon and resulted in bodily injury to the victim. A third officer, Deonte Pate, 23, was charged along with Marsher and Sturdivant for conspiring to cover up the beating. The indictment alleges that all three officers submitted false reports and that all three lied to the FBI. If convicted, Marsher and Sturdivant face a maximum sentence of 10 years in prison on the excessive force charges. Each of the three officers faces up to five years in prison on the conspiracy and false statement charges, and up to 20 years in prison on the false report charges. An indictment is merely an accusation, and the defendants are presumed innocent unless and until proven guilty. This case is being investigated by the FBI’s Jackson Division, with the cooperation of the Mississippi Department of Corrections. It is being prosecuted by Assistant U.S. Attorney Robert Coleman of the Northern District of Mississippi and Trial Attorney Dana Mulhauser of the Civil Rights Division’s Criminal Section. Marsher Indictment",nine count indict unseal today mississippi correct offic charg beat inmat third charg help cover indict charg lawardrick marsher robert sturdiv offic mississippi state penitentiari parchman mississippi beat includ kick punch throw victim ground marsher sturdiv charg violat right convict prison free cruel unusu punish sturdiv also charg fail interven marsher punch beat indict alleg action involv danger weapon result bodili injuri victim third offic deont pate charg along marsher sturdiv conspir cover beat indict alleg three offic submit fals report three lie convict marsher sturdiv face maximum sentenc year prison excess forc charg three offic face five year prison conspiraci fals statement charg year prison fals report charg indict mere accus defend presum innoc unless proven guilti investig jackson cooper mississippi correct prosecut robert coleman northern mississippi dana mulhaus crimin section marsher indict
313,16-217,"The Justice Department has reached a comprehensive settlement agreement with the city of Miami and the Miami Police Department (MPD) resolving the Justice Department’s investigation of officer-involved shootings by MPD officers, announced Principal Deputy Assistant Attorney General Vanita Gupta, head of the Justice Department’s Civil Rights Division and U.S. Attorney Wifredo A. Ferrer of the Southern District of Florida. The settlement, which was approved by Miami’s city commission today and will go into effect when the agreement is signed by all parties, resolves claims stemming from the Justice Department’s investigation into officer-involved shootings by MPD officers, which was conducted under the Violent Crime Control and Law Enforcement Act of 1994. The investigation’s findings, issued in July 2013, identified a pattern or practice of excessive use of force through officer-involved shootings in violation of the Fourth Amendment of the Constitution. The city’s compliance with the settlement will be monitored by an independent reviewer, former Tampa, Florida, Police Chief Jane Castor. Under the settlement agreement, the city will implement comprehensive reforms to ensure constitutional policing and support public trust. The settlement agreement is designed to minimize officer-inv

## 2.2 Create a document-term matrix from the preprocessed press releases and to explore top words (5 points)

A. Use the `create_dtm` function I provide (alternately, feel free to write your own!) and create a document-term matrix using the preprocessed press releases; make sure metadata contains the following columns: `id`, `compound` sentiment column you added, and the `topics_clean` column

B. Print the top 10 words for press releases with compound sentiment in the top 5% (so the most positive sentiment)

C. Print the top 10 words for press releases with compound sentiment in the bottom 5% (so the most negative sentiment)

**Hint**: for these, remember the pandas quantile function from pset two.  

D. Print the top 10 words for press releases in each of the three `topics_clean`

For steps B - D, to receive full credit, write a function `get_topwords` that helps you avoid duplicated code when you find top words for the different subsets of the data. There are different ways to structure it but one way is to feed it subsetted data (so data subsetted to one topic etc.) and for it to get the top words for that subset.

**Resources**:

- Here contains an example of applying the create_dtm function: 
https://github.com/rebeccajohnson88/PPOL564_slides_activities/blob/main/activities/fall_22/solutions/09_textasdata_partII_topicmodeling_solution.ipynb


In [19]:
def create_dtm(list_of_strings, metadata):
    vectorizer = CountVectorizer(lowercase = True)
    dtm_sparse = vectorizer.fit_transform(list_of_strings)
    dtm_dense_named = pd.DataFrame(dtm_sparse.todense(), 
        columns=vectorizer.get_feature_names())
    metadata.columns = ["metadata_" + col for col in metadata.columns]
    dtm_dense_named_withid = pd.concat([metadata.reset_index(), dtm_dense_named], axis = 1)
    return(dtm_dense_named_withid)

In [20]:
doj_subset_wscore_small = doj_subset_wscore.loc[~doj_subset_wscore.processed_text.isnull(),
           ['id', 'compound', 'processed_text','topics_clean']].copy().sample(n=700, random_state = 9899)

dtm_nopre = create_dtm(list_of_strings = doj_subset_wscore_small.processed_text,
                metadata = doj_subset_wscore_small[['id', 'compound', 'topics_clean']])

dtm_nopre.head()

,index,metadata_id,metadata_compound,metadata_topics_clean,aaron,abandon,abbat,abbi,abbott,abdomen,...,zane,zealand,zealou,zeeman,zero,zionism,zobel,zone,zunggeemog,zwengel
0,311,16-529,0.9727,Civil Rights,0,0,0,0,0,0,...,0,0,0,0,0,0,0,1,0,0
1,135,15-1290,-0.9524,Project Safe Childhood,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,37,17-1277,-0.9775,Hate Crimes,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,146,17-181,-0.9781,Civil Rights,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,120,17-160,-0.9744,Hate Crimes,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [21]:
## A.
def get_topwords(subsetted_list_of_string):
    term=subsetted_list_of_string[[col for col in subsetted_list_of_string.columns
                      if "metadata" not in col and
                      col != "index"]].sum(axis = 0)
    topwords=term.sort_values(ascending = False).head(n=10)
    return(topwords)

In [22]:
## B.Print the top 10 words for press releases with compound sentiment in the top 5%
dtm_nopre_top = dtm_nopre.loc[dtm_nopre.metadata_compound >= dtm_nopre.metadata_compound.quantile(0.95)]
get_topwords(dtm_nopre_top)

## C.Print the top 10 words for press releases with compound sentiment in the bottom 5%
dtm_nopre_bottom = dtm_nopre.loc[dtm_nopre.metadata_compound <= dtm_nopre.metadata_compound.quantile(0.05)]
get_topwords(dtm_nopre_bottom)

agreement     171
enforc        129
commun        114
ensur         108
state         108
disabl        107
court          89
student        87
servic         86
settlement     86
dtype: int64

assault     188
crime       170
victim      165
offic       155
hate        129
defend      126
conspir     115
american    103
sentenc     100
anderson     94
dtype: int64

In [23]:
# D. Print the top 10 words for press releases in each of the three 'topics_clean'
Civil_Rights=dtm_nopre.loc[dtm_nopre.metadata_topics_clean=='Civil Rights']
get_topwords(Civil_Rights)
Project_Safe_Childhood=dtm_nopre.loc[dtm_nopre.metadata_topics_clean=='Project Safe Childhood']
get_topwords(Project_Safe_Childhood)
Hate_Crimes=dtm_nopre.loc[dtm_nopre.metadata_topics_clean=='Hate Crimes']
get_topwords(Hate_Crimes)

offic        624
discrimin    602
hous         596
enforc       532
disabl       522
said         484
feder        468
violat       464
state        444
agreement    408
dtype: int64

child          993
exploit        689
sexual         560
safe           466
childhood      461
project        460
pornographi    433
children       417
crimin         396
prosecut       365
dtype: int64

victim      587
crime       555
hate        522
defend      479
prosecut    471
charg       458
sentenc     453
american    445
feder       425
guilti      424
dtype: int64

## 2.3 Estimate a topic model using those preprocessed words (5 points)

A. Going back to the preprocessed words from part 2.1, estimate a topic model with 3 topics, since you want to see if the unsupervised topic models recover different themes for each of the three manually-labeled topics (civil rights; hate crimes; project safe childhood). You have free rein over the other topic model parameters beyond the number of topics.

B. After estimating the topic model, print the top 15 words in each topic.

**Hints and Resources**:

- Same topic modeling resources linked to above
- Make sure to use the `random_state` argument within the model so that the numbering of topics does not move around between runs of your code

In [24]:
## A.
doj_subset_wscore = doj_subset_wscore[doj_subset_wscore.processed_text != ""].copy()

# Tokenize the processed_text;
tokenized_processed_text = [wordpunct_tokenize(one_text) 
                for one_text in 
                doj_subset_wscore.processed_text]

### create dictionary
text_proc_dict = corpora.Dictionary(tokenized_processed_text)

### filter dictionary- using 2% as bounds
text_proc_dict.filter_extremes(no_below = round(doj_subset_wscore.shape[0]*0.02),
                             no_above = round(doj_subset_wscore.shape[0]*0.98))

### create corpus from dictionary
corpus_fromdict_proc = [text_proc_dict.doc2bow(one_text) 
                       for one_text in tokenized_processed_text]

### estimate model
n_topics = 3
ldamod_proc = gensim.models.ldamodel.LdaModel(corpus_fromdict_proc, 
                                         num_topics = n_topics, id2word=text_proc_dict, 
                                         passes=6, alpha = 'auto',
                                         per_word_topics = True, random_state = 91988)

In [25]:
import pprint
pp = pprint.PrettyPrinter(indent=4)

topics = ldamod_proc.print_topics(num_words = 15)
for topic in topics:
    pp.pprint(topic[1])
    print("")

('0.017*"hous" + 0.016*"discrimin" + 0.015*"disabl" + 0.010*"agreement" + '
 '0.009*"state" + 0.008*"said" + 0.008*"enforc" + 0.008*"fair" + '
 '0.008*"settlement" + 0.008*"court" + 0.008*"requir" + 0.007*"individu" + '
 '0.007*"alleg" + 0.007*"employ" + 0.007*"inform"')

('0.013*"victim" + 0.013*"child" + 0.012*"sentenc" + 0.012*"prosecut" + '
 '0.011*"charg" + 0.010*"guilti" + 0.010*"feder" + 0.009*"year" + '
 '0.009*"investig" + 0.009*"exploit" + 0.008*"defend" + 0.008*"prison" + '
 '0.008*"sexual" + 0.008*"crime" + 0.008*"crimin"')

('0.012*"offic" + 0.011*"enforc" + 0.011*"religi" + 0.010*"commun" + '
 '0.009*"said" + 0.008*"gener" + 0.008*"feder" + 0.007*"counti" + '
 '0.007*"protect" + 0.007*"violat" + 0.007*"state" + 0.007*"court" + '
 '0.006*"school" + 0.006*"today" + 0.006*"polic"')



## 2.4 Add topics back to main data and explore correlation between manual labels and our estimated topics (10 points)

A. Extract the document-level topic probabilities. Within `get_document_topics`, use the argument `minimum_probability` = 0 to make sure all 3 topic probabilities are returned. Write an assert statement to make sure the length of the list is equal to the number of rows in the `doj_subset_wscores` dataframe

B. Add the topic probabilities to the `doj_subset_wscores` dataframe as columns and create a column, `top_topic`, that reflects each document to its highest-probability topic (eg topic 1, 2, or 3)

C. For each of the manual labels in `topics_clean` (Hate Crime, Civil Rights, Project Safe Childhood), print the breakdown of the % of documents with each top topic (so, for instance, Hate Crime has 246 documents-- if 123 of those documents are coded to topic_1, that would be 50%; and so on). 
**Hint**:    
pd.crosstab and normalize may be helpful: https://pandas.pydata.org/pandas-docs/version/0.23/generated/pandas.crosstab.html

D. Using a couple press releases as examples, write a 1-2 sentence interpretation of why some of the manual topics map on more cleanly to an estimated topic than other manual topic(s)

**Resources**:

- End of this code (`Additional summaries of topics and documents`) contains example of how to use `get_document_topics` and other steps to add topic probabilities back to data: 
https://github.com/rebeccajohnson88/PPOL564_slides_activities/blob/main/activities/fall_22/solutions/09_textasdata_partII_topicmodeling_solution.ipynb
- If you're getting errors, use shape, len, and other commands to check the dimensionality of things at different steps since documents may be dropped if they contain no words post-processing 

In [26]:
topic_probs_bydoc = [ldamod_proc.get_document_topics(item,minimum_probability=0) for item in corpus_fromdict_proc]
one_list_tup = topic_probs_bydoc[0]
assert_state = np.where(len(topic_probs_bydoc) == len(doj_subset_wscore), True, False)
assert_state

array(True)

In [27]:
## B.Add those topic probabilities to the dataframe
## create a long form dataframe by flattening the list
topic_probs_bydoc_long = pd.DataFrame([t for lst in topic_probs_bydoc for t in lst],
                                     columns = ['topic', 'probability'])

## add id var- we're repeating each id in the original data k times
## for the number of topics
topic_probs_bydoc_long['doc_id'] = list(np.concatenate([[one_id] * 
                                    n_topics for one_id in doj_subset_wscore.id]).flat)

## pivot to wide format
topic_probs_bydoc_wide = pd.pivot_table(topic_probs_bydoc_long, index = ['doc_id'],
                        columns = ['topic']).reset_index().reset_index(drop = True)
topic_probs_bydoc_wide.columns = ['doc_id'] + ["topic_" + str(i+1) for i in np.arange(0, n_topics)]

## merge with original data using doc id
topic_wmeta = pd.merge(topic_probs_bydoc_wide,
                      doj_subset_wscore,
                      left_on = 'doc_id',
                      right_on = 'id')

## create indicator for listing's top topic
topic_wmeta['top_topic'] = topic_wmeta[[col for col in topic_wmeta.columns if 
                                    "topic_" in col]].idxmax(axis=1)
topic_wmeta.sample(n = 5, random_state = 555)

doc_id   topic_1   topic_2   topic_3  index       id  \
259  15-1263  0.000359  0.874648  0.124993   4215  15-1263   
429   16-252  0.000611  0.563148  0.436242   3581   16-252   
374  16-1229  0.986450  0.012408  0.001142   7125  16-1229   
82   11-1328  0.001033  0.997941  0.001026   8491  11-1328   
239   15-015  0.000138  0.999725  0.000137  11763   15-015   

                                                                                                                                                                                title  \
259                                              Former Mamou, Louisiana, Police Chief Sentenced, Second Former Police Chief Pleads Guilty to Firing Taser at Non-Combative Prisoners   
429                                                                                          Former Alcorn State University Police Officer Pleads Guilty to Assaulting Former Student   
374  Justice Department Seeks to Intervene in Lawsuit Alleging Race Discrimination and Retaliation by Pocomoke City, Maryland, the Worcester County Sheriff and the State of Maryland   
82                                                                             Member of United Aryan Brotherhood Pleads Guilty to a Hate Motivated Assault of Jewish Inmate in Texas   
239                                                           Two Brandon, Mississippi, Men Plead Guilty for Committing Hate Crimes Against African Americans in Jackson, Mississippi   

                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                          

In [28]:
## C.Summarize the topic proportions for each of the topics_clean 
## group by topic
#topic_wmeta.groupby('topics_clean').agg({'top_topic': pd.crosstab})
pd.crosstab(topic_wmeta.topics_clean, topic_wmeta.top_topic, rownames=['topics_clean'], colnames=['top_topic'], normalize=True)

top_topic,topic_1,topic_2,topic_3
topics_clean,,,
Civil Rights,0.212164,0.118812,0.094767
Hate Crimes,0.000000,0.335219,0.012730
Project Safe Childhood,0.000000,0.223479,0.002829


This seems to indicate that Civil Rights is mostly topic_1 with a bit of bleed over as topic_2. But topic_2 seems to be a diverse blend of all three main subjects (but it's mainly comprised of Civil Rights, Project Safe Childhood, and then Civil Rights and in that order).



# 3. Extend the analysis from unigrams to bigrams (9 points)

In the previous question, you found top words via a unigram representation of the text. Now, we want to see how those top words change with bigrams (pairs of words)

A. Using the `doj_subset_wscore` data and the `processed_text` column (so the words after stemming/other preprocessing), create a column in the data called `processed_text_bigrams` that combines each consecutive pairs of word into a bigram separated by an underscore. Eg:

"depart reach settlem" would become "depart_reach reach_settlem"

Do this by writing a function `create_bigram_onedoc` that takes in a single `processed_text` string and returns a string with its bigrams structured similarly to above example
 
**Hint**: there are many ways to solve but `zip` may be helpful: https://stackoverflow.com/questions/21303224/iterate-over-all-pairs-of-consecutive-items-in-a-list

B. Print the `id`, `processed_text`, and `processed_text_bigram` columns for press release with id = 16-217

In [29]:
## A. Write a function to return bigrams structured
processed_text_list = doj_subset_wscore.processed_text.to_list()

# Write the function here
def create_bigram_onedoc(single_string):
    
    dictionary = {}
    i = 0
    # Split all the words
    single_string_split = single_string.split(' ')
    
    # Make all words in pair
    for first,second in zip(single_string_split,single_string_split[1:]):
        string_pair = first + '_' + second
        dictionary[i] = string_pair
        i += 1
    
    # make dictionary to list
    dic_list = list(dictionary.values())
    
    # connect the list
    s = ','
    list_connect = s.join(dic_list)
    list_space = list_connect.replace(',',' ')
    return(list_space)

doj_subset_wscore['processed_text_bigram'] = [create_bigram_onedoc(text) for text in doj_subset_wscore.processed_text]
doj_subset_wscore

,index,id,title,contents,date,topics_clean,components_clean,neg,neu,pos,compound,processed_text,processed_text_bigram
13,329,14-248,Albuquerque Man Charged with Federal Hate Crime Related to Anti-Semitic Threats Against Businesswoman,"The Department of Justice announced that this morning John W. Ng, 58, of Albuquerque, N.M., made his initial appearance in federal court on a criminal complaint charging him with a hate crime offense. This charge is related to anti-Semitic threats Ng made against a Jewish woman who owns and operates the Nosh Jewish Delicatessen and Bakery in Albuquerque. Ng was arrested by the FBI on March 7, 2014, based on a criminal complaint alleging that he interfered with the victim’s federally protected rights by threatening her and interfering with her business because of her religion. According to the criminal complaint, between Jan. 22, 2014, and Feb. 8, 2014, Ng allegedly posted threatening anti-Semitic notes on and in the vicinity of the victim’s business. A criminal complaint merely establishes probable cause, and Ng is presumed innocent unless proven guilty. If convicted on the offense charged in the criminal complaint, Ng faces a maximum statutory penalty of one year in prison. This matter was investigated by the Albuquerque Division of the FBI and is being prosecuted by Assistant U.S. Attorney Mark T. Baker of the U.S. Attorney’s Office for the District of New Mexico and Trial Attorney AeJean Cha of the U.S. Department of Justice’s Civil Rights Division.",2014-03-10T00:00:00-04:00,Hate Crimes,Civil Rights Division; Civil Rights - Criminal Section,0.315,0.654,0.031,-0.9951,announc morn john albuquerqu made initi appear feder court crimin complaint charg hate crime offens charg relat anti semit threat made jewish woman own oper nosh jewish delicatessen bakeri albuquerqu arrest march base crimin complaint alleg interf victim feder protect threaten interf busi religion accord crimin complaint allegedli post threaten anti semit note vicin victim busi crimin complaint mere establish probabl caus presum innoc unless proven guilti convict offens charg crimin complaint face maximum statutori penalti year prison matter investig albuquerqu prosecut mark baker mexico aejean,announc_morn morn_john john_albuquerqu albuquerqu_made made_initi initi_appear appear_feder feder_court court_crimin crimin_complaint complaint_charg charg_hate hate_crime crime_offens offens_charg charg_relat relat_anti anti_semit semit_threat threat_made made_jewish jewish_woman woman_own own_oper oper_nosh nosh_jewish jewish_delicatessen delicatessen_bakeri bakeri_albuquerqu albuquerqu_arrest arrest_march march_base base_crimin crimin_complaint complaint_alleg alleg_interf interf_victim victim_feder feder_protect protect_threaten threaten_interf interf_busi busi_religion religion_accord accord_crimin crimin_complaint complaint_allegedli allegedli_post post_threaten threaten_anti anti_semit semit_note note_vicin vicin_victim victim_busi busi_crimin crimin_complaint complaint_mere mere_establish establish_probabl probabl_caus caus_presum presum_innoc innoc_unless unless_proven proven_guilti guilti_convict convict_offens offens_charg charg_crimin crimin_complaint complaint_face face_maximum maximum_statutori statutori_penalti penalti_year year_prison prison_matter matter_investig investig_albuquerqu albuquerqu_prosecut prosecut_mark mark_baker baker_mexico mexico_aejean
632,11593,16-718,Three Mississippi Correctional Officers Indicted for Inmate Assault and Cover-Up,"In a nine-count indictment unsealed today, two Mississippi correctional officers were charged with beating an inmate and a third was charged with helping to cover it up. The indictment charged Lawardrick Marsher, 28, and Robert Sturdivant, 47, officers at Mississippi State Penitentiary, in Parchman, Mississippi, with a beating that included kicking, punching and throwing the victim to the ground. Marsher and Sturdivant were charged with violating the right of K.H., a convicted 

In [30]:
doj_subset_wscore.loc[doj_subset_wscore.id == '16-217'][['id','processed_text','processed_text_bigram']]

,id,processed_text,processed_text_bigram
313,16-217,reach comprehens settlement agreement citi miami miami polic resolv offic involv shoot offic announc princip deputi gener vanita gupta head wifredo ferrer southern florida settlement approv miami citi commiss today effect agreement sign parti resolv claim stem offic involv shoot offic conduct violent crime control enforc find issu juli identifi pattern practic excess forc offic involv shoot violat fourth amend constitut citi complianc settlement monitor independ review former tampa florida polic chief jane castor settlement agreement citi implement comprehens reform ensur constitut polic support public trust settlement agreement design minim offic involv shoot effect quickli investig offic involv shoot occur measur includ settlement repres renew commit citi miami chief rodolfo llane provid constitut polic miami resid protect public safeti sustain reform said princip deputi gener gupta agreement help strengthen relationship commun serv improv account offic fire weapon unlaw provid commun particip enforc agreement today agreement result joint effort citi miami ensur miami polic continu effort make commun safe protect sacr constitut citizen said ferrer oversight commun agreement seek make perman posit chang former chief orosa chief llane made applaud citi commiss vote settlement agreement build upon import reform implement citi sinc issu find includ conduct attorney staff special litig section southern florida,reach_comprehens comprehens_settlement settlement_agreement agreement_citi citi_miami miami_miami miami_polic polic_resolv resolv_offic offic_involv involv_shoot shoot_offic offic_announc announc_princip princip_deputi deputi_gener gener_vanita vanita_gupta gupta_head head_wifredo wifredo_ferrer ferrer_southern southern_florida florida_settlement settlement_approv approv_miami miami_citi citi_commiss commiss_today today_effect effect_agreement agreement_sign sign_parti parti_resolv resolv_claim claim_stem stem_offic offic_involv involv_shoot shoot_offic offic_conduct conduct_violent violent_crime crime_control control_enforc enforc_find find_issu issu_juli juli_identifi identifi_pattern pattern_practic practic_excess excess_forc forc_offic offic_involv involv_shoot shoot_violat violat_fourth fourth_amend amend_constitut constitut_citi citi_complianc complianc_settlement settlement_monitor monitor_independ independ_review review_former former_tampa tampa_florida florida_polic polic_chief chief_jane jane_castor castor_settlement settlement_agreement agreement_citi citi_implement implement_comprehens comprehens_reform reform_ensur ensur_constitut constitut_polic polic_support support_public public_trust trust_settlement settlement_agreement agreement_design design_minim minim_offic offic_involv involv_shoot shoot_effect effect_quickli quickli_investig investig_offic offic_involv involv_shoot shoot_occur occur_measur measur_includ includ_settlement settlement_repres repres_renew renew_commit commit_citi citi_miami miami_chief chief_rodolfo rodolfo_llane llane_provid provid_constitut constitut_polic polic_miami miami_resid resid_protect protect_public public_safeti safeti_sustain sustain_reform reform_said said_princip princip_deputi deputi_gener gener_gupta gupta_agreement agreement_help help_strengthen strengthen_relationship relationship_commun commun_serv serv_improv improv_account account_offic offic_fire fire_weapon weapon_unlaw unlaw_provid provid_commun commun_particip particip_enforc enforc_agreement agreement_today today_agreement agreement_result result_joint joint_effort effort_citi citi_miami miami_ensur ensur_miami miami_polic polic_continu continu_effort effort_make make_commun commun_safe safe_protect protect_sacr sacr_constitut constitut_citizen citizen_said said_ferrer ferrer_oversight oversight_commun commun_agreement agreement_seek seek_make make_perman perman_posit posit_chang chang_former former_chief chief_orosa orosa_chief chief_llane llane_made made_app

C. Use the create_dtm function and the `processed_text_bigrams` column to create a document-term matrix (`dtm_bigram`) with these bigrams. Keep the following three columns in the data: `id`, `topics_clean`, and `compound` 

D. Print the 
 (1) dimensions of the `dtm` matrix from question 2.2  and 
 (2) the dimensions of the `dtm_bigram` matrix. Comment on why the bigram matrix has more dimensions than the unigram matrix 

In [31]:
## C. 
doj_subset_wscore_bigram = doj_subset_wscore.loc[~doj_subset_wscore.processed_text_bigram.isnull(),
           ['id', 'compound', 'processed_text_bigram','topics_clean']].copy().sample(n =700, random_state = 9899)

 
dtm_bigram = create_dtm(list_of_strings= doj_subset_wscore_bigram.processed_text_bigram,
                metadata = 
                doj_subset_wscore_bigram[['id', 'compound', 'topics_clean']])

dtm_bigram.head()

,index,metadata_id,metadata_compound,metadata_topics_clean,aaron_ford,aaron_latham,aaron_mcgrath,aaron_parrish,aaron_polster,aaron_rice,...,zone_ordin,zone_practic,zone_religi,zone_restrict,zone_student,zone_varianc,zunggeemog_noel,zunggeemog_prompt,zunggeemog_write,zwengel_princeton
0,311,16-529,0.9727,Civil Rights,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,135,15-1290,-0.9524,Project Safe Childhood,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,37,17-1277,-0.9775,Hate Crimes,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,146,17-181,-0.9781,Civil Rights,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,120,17-160,-0.9744,Hate Crimes,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [32]:
## D. Print the dimension

# Dimension of dtm_nopre in 2.2
dtm_nopre.shape
# Dimension of dtm_bigram
dtm_bigram.shape

(700, 6881)

(700, 71717)

It makes sense that there would be more dimensions because pairs of words will be higher than the raw number of words.

E. Find and print the 10 most prevelant bigrams for each of the three topics_clean using the `get_topwords` function from 2.2


In [33]:
## E. Print the 10 most prevelant bigrams for each of the three topics_clean
Civil_Rights_bigram = dtm_bigram.loc[dtm_bigram.metadata_topics_clean == 'Civil Rights']
get_topwords(Civil_Rights_bigram)
Project_Safe_Childhood_bigram = dtm_bigram.loc[dtm_bigram.metadata_topics_clean == 'Project Safe Childhood']
get_topwords(Project_Safe_Childhood_bigram)
Hate_Crimes_bigram = dtm_bigram.loc[dtm_bigram.metadata_topics_clean == 'Hate Crimes']
get_topwords(Hate_Crimes_bigram)

fair_hous         219
deputi_gener      214
princip_deputi    214
vanita_gupta      195
gupta_head        193
gener_vanita      192
said_princip      180
unit_state        153
nation_origin     141
consent_decre     126
dtype: int64

safe_childhood       461
project_safe         460
child_pornographi    431
child_exploit        277
sexual_exploit       219
exploit_children     196
plead_guilti         191
exploit_obscen       172
obscen_section       171
child_sexual         170
dtype: int64

hate_crime          378
african_american    361
plead_guilti        271
year_prison         158
special_agent       124
racial_motiv        114
thoma_perez         109
grand_juri          100
perez_gener          93
said_thoma           89
dtype: int64

# 4. Optional extra credit 1 (1 point)

You notice that the pharmaceutical kickbacks press release we analyzed in question 1 was for an indictment, and that in the original data, there's not a clear label for whether a press release outlines an indictment (charging someone with a crime), a conviction (convicting them after that charge either via a settlement or trial), or a sentencing (how many years of prison or supervised release a defendant is sentenced to after their conviction).

You want to see if you can identify pairs of press releases where one press release is from one stage (e.g., indictment) and another is from a different stage (e.g., a sentencing).

You decide that one way to approach is to find the pairwise string similarity between each of the processed press releases in `doj_subset`. There are many ways to do this, so Google for some approaches, focusing on ones that work well for entire documents rather than small strings.

Find the top two pairs (so four press releases total)-- do they seem like different stages of the same crime or just press releases covering similar crimes?

In [ ]:
## your code here

# 5. Optional extra credit 2 (3 points)

Review the scraping code here: https://github.com/jbencina/dojreleases/blob/master/scraper.py
    
Write code to scrape press releases from more recent years from the DOJ website than the years available in the combined.json and produce a visualization of how top words or themes in the press releases changed across the Trump administration (2016-December 2020) versus Biden administration (January 2021 onwards)- you can do this in a .py file that you submit separately and just read in the data produced by that scraping

In [ ]:
## your code here